In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar2/"
use_tfidfvectorizer = False
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 30
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.5.1s"
merge_list = ["t2.5.2", "t2.5.4", "t2.5.6", "t2.5.3"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.1s.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

218378

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.2.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.4.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.6.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.3.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

218378

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 218378


topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

17.704392194747925

In [18]:
word_doc_matrix

<218378x9599 sparse matrix of type '<class 'numpy.int64'>'
	with 37224704 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

Epochs completed:   0%|            0/200 [00:00]

CPU times: user 1h 47min 7s, sys: 5min 25s, total: 1h 52min 32s
Wall time: 22min 16s


In [21]:
print(runtime_umap)

1336.3279712200165


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 4h 49min 21s, sys: 49.1 s, total: 4h 50min 10s
Wall time: 4h 46min 23s


In [23]:
print(runtime_umap_transform)

17183.233987808228


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//umap_embedding_t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//umap_clusterable_embedding_t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs450,450,50,4.023905,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
73,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs450,450,100,4.343788,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs450,450,150,4.686385,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs450,450,200,5.266787,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs450,450,100,4.343788,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs450,450,150,4.686385,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs450,450,200,5.266787,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs450,450,250,5.780266,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs450,450,150,4.686385,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs450,450,200,5.266787,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs450,450,250,5.780266,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss300cs450,450,300,6.659279,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs450,450,200,5.266787,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs450,450,250,5.780266,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss300cs450,450,300,6.659279,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
78,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss350cs450,450,350,6.970911,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss50cs50,50,50,4.256437,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss100cs50,50,100,4.111580,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss150cs50,50,150,4.764697,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss200cs50,50,200,5.322671,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs50,50,250,5.816087,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss250cs450,450,250,5.780266,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss300cs450,450,300,6.659279,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
78,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss350cs450,450,350,6.970911,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."
79,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,ss400cs450,450,400,7.754092,"rt_vect: 17.704392194747925, min_df: 4, max_df...","hellinger, rt_umap_tf: 17183.233987808228, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,technopagan/mss-saliency,59,45,30,14,12,10,-1,4,4,...,3,-1,9,5,2,1,-1,-1,2,2
1,snmnmin12/selective-search,182,141,73,59,37,30,4,4,4,...,3,3,9,6,2,1,3,2,2,2
2,google/chaps-linux,-1,-1,91,-1,34,19,14,4,4,...,3,3,9,5,2,1,5,7,2,2
3,lukasolson/speech-router,-1,-1,-1,-1,34,19,14,4,4,...,3,3,9,5,2,1,5,7,2,2
4,XavierTecBr/qbr-admin,61,44,29,19,-1,-1,-1,4,4,...,3,-1,9,6,2,1,-1,-1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,53,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,2,1,-1,-1,-1,-1
218374,LandGrey/SpringBootVulExploit,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
218375,mucomplex/Pentest_checklist,156,88,71,60,34,19,14,4,4,...,3,3,9,5,2,1,5,7,2,2
218376,bigsen/Vision_Track,-1,-1,-1,-1,-1,-1,4,4,4,...,3,3,9,6,2,1,3,2,2,2


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,0,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.574631,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,0,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.574631,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,0,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.889588,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,0,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.574631,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,0,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.889588,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,0,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",2.501532,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,0,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.574631,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,0,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.889588,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,0,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",2.501532,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
5,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,25,0,25,"rt_vect: 17.704392194747925, min_df: 4, max_df...",2.861945,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,0,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.202722,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,0,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.366034,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,0,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.574631,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,0,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",1.889588,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,0,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",2.501532,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
5,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,25,0,25,"rt_vect: 17.704392194747925, min_df: 4, max_df...",2.861945,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."
6,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,28,0,28,"rt_vect: 17.704392194747925, min_df: 4, max_df...",3.335046,"(hellinger, rt_umap_tf: 17183.233987808228, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,7,8,11,8,2,13,19
1,snmnmin12/selective-search,3,3,13,1,10,11,9
2,google/chaps-linux,2,11,9,2,12,16,13
3,lukasolson/speech-router,8,1,0,3,9,15,10
4,XavierTecBr/qbr-admin,3,3,1,11,16,11,26
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,3,3,1,1,16,11,26
218374,LandGrey/SpringBootVulExploit,4,4,15,12,4,1,7
218375,mucomplex/Pentest_checklist,2,11,10,5,5,2,4
218376,bigsen/Vision_Track,3,3,1,1,16,11,9


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",304.856906


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",304.856906
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",453.178225


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",304.856906
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",453.178225
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",590.426109


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",304.856906
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",453.178225
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",590.426109
5,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,25,25,"rt_vect: 17.704392194747925, min_df: 4, max_df...",632.788220


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,10,10,"rt_vect: 17.704392194747925, min_df: 4, max_df...",143.266361
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,13,13,"rt_vect: 17.704392194747925, min_df: 4, max_df...",192.153064
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,16,16,"rt_vect: 17.704392194747925, min_df: 4, max_df...",304.856906
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,19,19,"rt_vect: 17.704392194747925, min_df: 4, max_df...",453.178225
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,22,22,"rt_vect: 17.704392194747925, min_df: 4, max_df...",590.426109
5,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,25,25,"rt_vect: 17.704392194747925, min_df: 4, max_df...",632.788220
6,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3,1,28,28,"rt_vect: 17.704392194747925, min_df: 4, max_df...",807.324766


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,2,8,15,7,2,15,4
1,snmnmin12/selective-search,7,1,12,1,14,3,6
2,google/chaps-linux,8,7,1,13,1,9,1
3,lukasolson/speech-router,7,8,10,8,2,4,27
4,XavierTecBr/qbr-admin,7,1,12,1,14,3,6
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,7,1,12,1,14,3,6
218374,LandGrey/SpringBootVulExploit,2,7,15,13,16,9,1
218375,mucomplex/Pentest_checklist,2,8,15,7,16,9,4
218376,bigsen/Vision_Track,7,1,12,1,14,3,6


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6+2.5.3.pkl
